In [ ]:
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['font.size'] = 12
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, widgets
from utils import riemann_tools
from exact_solvers import traffic_variable_speed

# LWR Traffic flow with varying speed limit

In this chapter we again consider the LWR traffic model, but now we introduce a speed limit $u_\text{max}$:

$$
q_t + (u_\text{max}(x) q(1-q))_x = 0.
$$

In the previous chapter, the speed limit was set to unity everywhere.  The variable-speed-limit case has been considered, for instance, in <cite data-cite="mochon1987"><a href="riemann.html#mochon1987">(Mochon, 1987)<a></cite> and in Chapter 16 of <cite data-cite="fvmhp"><a href="riemann.html#fvmhp">(LeVeque, 2002)<a></cite>.  From a physical point of view, we might also imagine that $u_\text{max}$ varies because of differing road conditions -- for instance, if part of the road is wet or foggy.

Now we consider the Riemann problem with different speed limits to the left and right:

$$
u_\text{max} = \begin{cases} v_l & x<0 \\ v_r & x>0 \end{cases}
$$


Correspondingly, we have a flux function that is different on either side of $x=0$.  Let $f(q,v) = v q (1-q)$ and define

\begin{align}
    f_l & = f(q_l,v_l) \\
    f_r & = f(q_r,v_r).
\end{align}

The presence of a discontinuous flux function in the Riemann problem can introduce a host of difficulties, some of which will be illustrated here.  For a more detailed discussion, see <cite data-cite="Burger2008"><a href="riemann.html#fvmhp">(Burger et. al., 2008)<a></cite> and other papers in that special issue.

Continuing with the problem at hand, if, say, $v_l=1$ and $v_r=2$ then the two flux functions look like this: 

In [ ]:
q = np.linspace(0,1)
f_l = q*(1-q); f_r = 2*q*(1-q)
plt.plot(q,f_l,q,f_r); plt.xlim(0,1); plt.ylim(0,0.6);
plt.xlabel('q',fontsize=20); plt.ylabel('f(q)',fontsize=20)
plt.legend(['$f_l$','$f_r$'],fontsize=20);

This leads to a more complicated set of possible Riemann solutions.  As discussed at length in the previous chapter, the flux of vehicles must be continuous everywhere -- and in particular, at $x=0$:
$$
f(q(0^-,t),v_l)) = f(q(0^+,t),v_r)) \equiv f^*.
$$

If the speed limit changes abruptly at $x=0$, then (in order to have a continuous flux) the density must be discontinuous there.  Since there are two possible values of $q$ corresponding to any given flux, we must carefully determine the intermediate states.  We can think of the discontinuity at $x=0$ as an additional wave with speed zero.

Furthermore, in the variable-speed-limit case, the Riemann solution may have a wave (shock or rarefaction) traveling in each direction.  Thus the solution in general may consist of a left-going wave, a stationary jump, and a right-going wave.  This means that in general we may refer to four states:

1. $q_l$;
2. $q_l^*$, connected to $q_l$ by a left-going shock or rarefaction;
3. $q_r^*$, connected to $q_r$ by a right-going shock or rarefaction;
4. $q_r$.

It often happens that either $q_l^*=q_l$ or $q_r^*=q_r$.  We can have $q_r^*=q_l^*$ only in the fixed speed limit case $v_l=v_r$.

Since the flux is continuous at $x=0$, the stationary jump there corresponds to a horizontal line in the $q-f(q)$ plane plotted above.  This jump connects a point on one flux curve to a point on the other.  Two things are immediately clear:

1. The value of $f^*$ must not lie fully above either of the flux curves; since the maximum flux is $v/4$, this means that $f^* \le \min(v_l,v_r)/4$.  If this condition were violated, it would be impossible to connect the two flux curves with a horizontal line.
2. For a given $f^*$ satisfying condition 1, there are always **two** values of $q$ on each curve with flux $f^*$.  This means that if (say) we have determined the value $q(x=0^-,t)$, there are two candidate values of $q(x=0^+,t)$ -- one with positive characteristic velocity, and one with negative characteristic velocity.

Physical reasoning can be used to determine which of the values is correct, but there is also a simple rule that tells us which value to choose in most cases:

> The sign of the characteristic velocity as one approaches $x=0$ is (in a weak sense) the same from either side.  That is, $f_l'(q(x=0^-,t)) f_r'(q(x=0^+,t)) \ge 0$.

If we know the value of $q$ just to one side of $x=0$, this condition tells us which value of $q$ to choose for the other side, except when the characteristic velocity on the side we know approaches zero (i.e., when $q$ approaches $1/2$.  Geometrically, the condition says that the horizontal line corresponding to the stationary jump must not cross the line $q=1/2$ in the plane $q-f(q)$ plane (though it may start or end on that line).  In the exceptional case, we can still determine which value to jump to by considering the sign of the characteristic velocity near $x=0$.  We  illustrate this further through specific examples below.

For now, we illustrate the division of the $q-f(q)$ plane based on the sign of the characteristic velocity.

In [ ]:
q = np.linspace(0,1)
f_l = q*(1-q); f_r = 2*q*(1-q)
plt.plot(q,f_l,q,f_r); plt.xlim(0,1); plt.ylim(0,0.6);
plt.xlabel('q',fontsize=20); plt.ylabel('f(q)',fontsize=20)
plt.plot([0.5,0.5],[0,0.6],'--k',lw=1,alpha=0.5);
plt.text(0.1,0.5,'$c>0$',fontsize=20);
plt.text(0.8,0.5,'$c<0$',fontsize=20);

Since the value $q=1/2$ plays such a crucial role, we will say a section of road is *congested* if $q$ is greater than $1/2$ there.

Finally, let us formalize one thing that we observed in the previous notebook:

>In the LWR system, shocks always carry a increase in density (from left to right), while rarefactions always carry a decrease in density.

Why is this?  The Lax entropy condition for a shock moving at speed $s$ tells us

$$f'(q_l) > s > f'(q_r).$$

In partcular, $f'(q_l) > f'(q_r)$, which for LWR means $1-2q_l > 1-2q_r$, or $q_r > q_l$.  This guarantees that characteristics are converging in the neighborhood of the shock.  At a rarefaction, characteristics must be diverging, so we must have the opposite inequality: $q_r < q_l$.

## Downstream congestion

If $q_r>1/2$, we expect a wave traveling to the left.  If we suppose there is only one wave, then conservation can tell us whether it should be a shock or a rarefaction.  As we just observed, in the LWR system, shocks always carry an increase in density (from left to right), while rarefactions always carry a decrease in density.  Hence a left-moving shock corresponds to a local increase in mass and is physical only if $f_l>f_r$.  Meanwhile, a left-moving rarefaction corresponds to a local decrease in mass and makes sense only if $f_l<f_r$.  In either case, in addition to the shock or rarefaction there is a jump in density at $x=0$.

### $f_l>f_r$: Left-going shock
Physically, there is a traffic jam expanding upstream; when drivers get through the jam and reach $x=0$, the congestion is lower.

In [ ]:
def c(q, x):
    if x<0:
        return v_l*(1.-2*q)
    else:
        return v_r*(1.-2*q)
    
def make_plot_function(q_l,q_r,v_l,v_r):
    states, speeds, reval, wave_types = traffic_variable_speed.exact_riemann_solution(q_l,q_r,v_l,v_r)
    def plot_function(t):
        ax = riemann_tools.plot_riemann(states,speeds,reval,wave_types,t=t,t_pointer=0,
                                        extra_axes=True,conserved_variables=['Density']);
        riemann_tools.plot_characteristics(reval,c,ax[0])
        traffic_variable_speed.phase_plane_plot(q_l,q_r,v_l,v_r,axes=ax[2])
        plt.show()
    return plot_function

def plot_riemann_traffic_vc(q_l,q_r,v_l,v_r):
    plot_function = make_plot_function(q_l,q_r,v_l,v_r)
    interact(plot_function, t=widgets.FloatSlider(value=0.5,min=0,max=.9))

In [ ]:
q_l = 0.7; q_r = 0.6
v_l = 1.0; v_r = 0.8
plot_riemann_traffic_vc(q_l,q_r,v_l,v_r);

It is illustrative to consider this solution in the plane whose axes are $q$ and $f$, shown in the rightmost plot above.  The Riemann solution is indicated with a thick line; as usual we indicate shocks in red and rarefactions in blue; the jump at $x=0$ is plotted in black.  Notice that the jump at the interface corresponds to a horizontal line, since the flux is the same on both sides of the interface.  Notice also that the jump does not cross the dashed line at $q=1/2$.

### f_l < f_r: Left-going rarefaction

In [ ]:
q_l = 0.8; q_r = 0.7
v_l = 1.1; v_r = 1.0
plot_riemann_traffic_vc(q_l,q_r,v_l,v_r);

Again, the jump at the interface does not cross the middle line.

## Sudden increase in speed limit
Looking at the $q-f$ plane plot above, it is natural to ask what happens if we increase the flux $f_r$ far enough that it lies above the maximum of the left-segment flux curve.  Then it becomes impossible to connect $q_r$ to a state on the left-segment flux curve by a horizontal line.  Physically, this corresponds to the situation in which the downstream flux is so high that the influx of cars from upstream cannot possibly keep up.  The downstream density must decrease, and in this case there will be not only a left-going rarefaction but also a right-going shock.

This occurs whenever

$$
f_r > v_l/4.
$$



In [ ]:
q_l = 0.7; q_r = 0.6
v_l = 1.0; v_r = 2.
plot_riemann_traffic_vc(q_l,q_r,v_l,v_r);

As promised, we see three waves: a left-going rarefaction as the upstream congestion decreases since downstream traffic flux is greater; a stationary jump at the interface, where cars instantaneously accelerate due to the higher speed limit; and a right-going shock carrying a drop in density (from right to left) as the road clears out since there is less traffic arriving from upstream.

In the phase plane we see that the rarefaction raises the density as far as it can go on the lower flux curve.  In other words, the flux from the left at the interface is as high as it can possibly be.  Since the characteristic speed just to the left of $x=0$ approaches zero, the condition introduced in the introduction does not tell us which value on the $f_r$ curve we should jump to.  Instead, we note that if the density increased at $x=0$, then $q^*_r$ would have to be connected to $q_r$ by a (*right-going*) rarefaction in which all characteristics are *left-going*, which makes no sense.

Instead, we see that $q^*_r$ and $q_r$ are connected by a *transonic shock*, i.e. a shock for which the impinging characteristic speeds from right and left have opposite signs.  Transonic shocks cannot occur in the fixed speed limit LWR model.

## A fictitious solution
In the previous example, you might wonder why there is an intermediate state with $c>0$ when both $q_r$ and $q_l$ have $c<0$.  In order to connect these two states without such an intermediate state, we can imagine imposing only a jump at the interface and a right-going shock or rarefaction.  Here is what we get by imposing a shock:

In [ ]:
states = np.array([[q_l,0.88,q_r]])
speeds = np.array([0,0.5])
def reval(xi):
    q = np.ones((1,len(xi)))
    return q
wave_types = ['contact','shock']
fig, ax = plt.subplots(1,2,figsize=(12,4))
traffic_variable_speed.phase_plane_plot(q_l,q_r,v_l,v_r,states,speeds,reval,wave_types,axes=ax[1],show=False)
riemann_tools.plot_waves(states,speeds,reval,wave_types,ax=ax[0],t_pointer=False,t=0)
riemann_tools.plot_characteristics(reval,c,ax[0])

As we can see, the shock is unphysical.  The characteristics to the left of the shock are emerging from it, rather than impinging on it; this violates the entropy condition.  It's also easy to see why we can't replace that shock with a rarefaction: the rarefaction would be right-going, but all the neighboring characteristics are left-going!

## Sudden decrease in speed limit
In the scenario above, we saw that there were not enough cars arriving to $x=0$ from the left to keep up with the number leaving to the right.  We can have just the opposite situation if

$$f_l > v_r/4.$$

In this case the flux of cars arriving at $x=0$ from the left is greater than the maximum possible flux of cars leaving from $x=0$ to the right.  Intuitively, this suggests that cars must be accumulating at $x=0$ (and then backing up behind those, so we expect a shock wave propagating to the left.

If the road is congested downstream ($q_r>1/2$), then we have (again) only this shock and a stationary jump to lower density:

In [ ]:
q_l = 0.7; q_r = 0.6
v_l = 1.0; v_r = 0.4
plot_riemann_traffic_vc(q_l,q_r,v_l,v_r);

However, something more intersting can happen if the road downstream is not congested (i.e., if $q_r<1/2$.  Mathematically, we see that in this case characteristics in the right half-plane travel to the right.  Since there is a high density of cars at $x=0$, we might expect a rarefaction as they spread out into the uncongested section of road.  That is indeed what happens:

In [ ]:
q_l = 0.7; q_r = 0.1
v_l = 1.0; v_r = 0.4
plot_riemann_traffic_vc(q_l,q_r,v_l,v_r);

This may seem counterintuitive at first, but is a familiar experience.  For instance, when a highway narrows to fewer lanes, it often happens that a traffic jam forms in the region just before where it narrows.  Just as one reaches the point where it narrows, the jam clears and one is able to accelerate because the next section of road is less congested (even though it is narrower).

## Transonic rarefaction
As in the fixed-speed-limit case, a transonic rarefaction can only occur if the characteristics on both sides are pointing away from the interface:

\begin{align}
f'(q_l) & \le 0 \\
f'(q_r) & \ge 0.
\end{align}
However, in the variable-speed-limit case there are additional conditions that must be satisfied.  Namely, the left and right fluxes cannot be so great as to be impossible to balance with the appropriate flux on the other side; i.e. we require
\begin{align}
f_l & \le v_r/4 \\
f_r & \le v_l/4.
\end{align}
This guarantees that neither the left nor right state lies above the entire (right or left) flux curve in the phase plane.

There is an additional wrinkle: as usual, we have a stationary jump in $q$ in order to maintain flux continuity at the interface.  Thus the transonic rarefaction is broken into two rarefactions, with an intermediate constant state lying either just to the left or just to the right of $x=0$.  Where this state lies depends on whether the speed limit increases or decreases at the interface.

### Transonic rarefaction with $v_l > v_r$

In [ ]:
q_l = 0.9; q_r = 0.2
v_l = 1.0; v_r = 0.8
plot_riemann_traffic_vc(q_l,q_r,v_l,v_r);

In this case, the characteristic speed approaches zero as we approach $x=0$ from the right.  We can see that $q^*_l$ must lie to the right of the dashed line, since otherwise the characteristic taking the value $q^*_l$ would be traveling to the right!

### Transonic rarefaction with $v_r > v_l$

In [ ]:
q_l = 0.9; q_r = 0.2
v_l = 1.0; v_r = 1.2
plot_riemann_traffic_vc(q_l,q_r,v_l,v_r);

## Other solutions
The remaining types of solutions are mirror images of those above, including a single right-going shock or rarefaction, or a combination left-going shock and right-going rarefaction.  In total there are 8 possible Riemann solution structures -- twice as many as for a constant-coefficient convex scalar conservation law.  Experiment with the sliders and plot below to find left and right states that lead to these additional possibilities.

In [ ]:
def plot_all(q_l,q_r,v_l,v_r):
    states, speeds, reval, wave_types = traffic_variable_speed.exact_riemann_solution(q_l,q_r,v_l,v_r)
    ax = riemann_tools.plot_riemann(states,speeds,reval,wave_types,t=0.5,extra_axes=True);
    riemann_tools.plot_characteristics(reval,c,ax[0])
    traffic_variable_speed.phase_plane_plot(q_l,q_r,v_l,v_r,axes=ax[2])
    plt.show()
    
interact(plot_all,
            q_l = widgets.FloatSlider(min=0.,max=1.,step=0.01,value=0.4),
            q_r = widgets.FloatSlider(min=0.,max=1.,step=0.01,value=0.7),
            v_l = widgets.FloatSlider(min=0.1,max=2.,value=1.),
            v_r = widgets.FloatSlider(min=0.1,max=2.,value=0.4),
        );

# Approximate solvers

In order to define a numerical solver, we need to compute fluctuations (for the first-order solver) as well as waves and speeds (for the second-order corrections).

### Fluctuations
It is straightforward to compute the fluctuations by first determining the flux at the interface (i.e., the Godunov flux) $f^*$.  We can then determine the fluctuations via

\begin{align}
    {\mathcal A}^-\Delta Q & = f^* - f_l \\
    {\mathcal A}^+\Delta Q & = f_r - f^*.
\end{align}

In fact, $f^*$ is always equal to one of $\{f_l, f_r, v_l/4, v_r/4\}$.  We can use the criteria discussed above to determine which of these possibilities is correct.

### Second-order corrections
The fluctuations above are the exact fluctuations from the true solution of the Riemann problem.  We could take a similar approach and include the exact wave(s) appearing in the Riemann solution in order to form second-order corrections.  This may include one or two waves (we need not include the stationary jump at the interface since it doesn't modify the value of the solution in either cell).  However, a simpler approach discussed in <cite data-cite="fvmhp"><a href="riemann.html#fvmhp">(LeVeque, 2002)<a></cite> seems to work well.  We use a single wave with strength $q_r-q_l$ and speed equal to the Rankine-Hugoniot average speed $(c_r + c_l)/2$.

In [ ]:
from clawpack import pyclaw
from clawpack import riemann

In [ ]:
def test_solver(q_l, q_r, v_l, v_r, riemann_solver):
    solver = pyclaw.ClawSolver1D(riemann_solver)

    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    solver.aux_bc_lower[0] = pyclaw.BC.extrap
    solver.aux_bc_upper[0] = pyclaw.BC.extrap

    x = pyclaw.Dimension(-1.0,1.0,50,name='x')
    domain = pyclaw.Domain(x)
    num_aux = 1
    state = pyclaw.State(domain,solver.num_eqn,num_aux)

    grid = state.grid
    xc=grid.p_centers[0]

    state.q[0,:] = q_l*(xc<0) + q_r*(xc>=0.)
    state.aux[0,:] = v_l*(xc<0) + v_r*(xc>=0.)  # Speed limit

    claw = pyclaw.Controller()
    claw.tfinal = 1.0
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.keep_copy = True
    claw.verbosity = 0

    claw.run()
    return xc, claw.frames

In [ ]:
q_l = 0.7; q_r = 0.5
v_l = 1.0; v_r = 1.5
t = 0.5

states, speeds, reval, wave_types = traffic_variable_speed.exact_riemann_solution(q_l,q_r,v_l,v_r)
x, frames = test_solver(q_l,q_r,v_l,v_r,riemann.traffic_vc_1D)

def plot_frame(t):
    ax = riemann_tools.plot_riemann(states, speeds, reval, wave_types, t,layout='horizontal',extra_axes=True);
    riemann_tools.plot_characteristics(states,speeds,c,ax[0])
    q = frames[int(t*10)].q[0,:]
    ax[1].plot(x,q,'-sg')
    ax[1].legend(['Exact','Approx'],loc=1);
    traffic_variable_speed.phase_plane_plot(q_l,q_r,v_l,v_r,axes=ax[2])
    plt.show()
    
interact(plot_frame,t=(0,1,0.1));

### $f$-wave solvers
The use of the full jump $q_r-q_l$ in the wave discussed above may seem strange since part of this jump remains stationary at the interface (and part of it may be sent in the opposite direction!)  For systems where the flux depends explicitly on $x$, it is often simplest to use the $f$-wave approach; for a scalar problem this corresponds to using the jump in the flux, $f_r-f_l$ in place of the product $s(q_r-q_l)$.

This technique can be used in the present system, but it has a small drawback.  At the center of a transonic rarefaction, the cell average is $1/2$ and the $f$-wave $f_r-f_l$ will be essentially zero.  This causes the limiter to activate and reduce the method locally to first order, with the consequence that the solution is inaccurate near the sonic point.  The first approach discussed above (where the wave strength is $q_r-q_l$) is free from this deficiency.

In [ ]:
q_l = 0.7; q_r = 0.2
v_l = 1.0; v_r = 1.0
t = 0.5

states, speeds, reval, wave_types = traffic_variable_speed.exact_riemann_solution(q_l,q_r,v_l,v_r)
x, frames  = test_solver(q_l,q_r,v_l,v_r,riemann.traffic_vc_1D)
x ,fframes = test_solver(q_l,q_r,v_l,v_r,riemann.traffic_vc_fwave_1D)

def plot_frame(t):
    q = frames[int(t*10)].q[0,:]
    q2 = fframes[int(t*10)].q[0,:]
    ax = riemann_tools.plot_riemann(states, speeds, reval, wave_types, t, layout='horizontal', extra_axes=True);
    riemann_tools.plot_characteristics(states,speeds,c,ax[0])
    ax[1].plot(x,q,'-sg')
    ax[1].plot(x,q2,'-or')
    ax[1].legend(['Exact','Approx','$f$-wave'],loc=3);
    ax[1].set_xlim(-0.2,0.2)
    traffic_variable_speed.phase_plane_plot(q_l,q_r,v_l,v_r,axes=ax[2])
    plt.show()
    
interact(plot_frame,t=(0,1,0.1));